# 2D Truss Elements

In this notebook we implement our first two-dimensional finite element code, focusing on an implementation of simple truss elements.

The structure of the code can later be converted into a general program to consider a variety of finite element formulations by modifying the appropriate subroutines to represent the force-displacement (stiffness) equations for each element.

**_NOTE:  
The figures, much of the structure and specific problems solved in this notebook are taken from the notebook:  
[13_truss_elements.ipynb](https://nbviewer.jupyter.org/github/jacojvr/Introductory-Finite-Elements/blob/a1431adaaaf5304a233e33e8967bde598790db9d/notebooks/13_truss_elements.ipynb) by Juan Gomez_**

In [1]:
# first import the mdules we'll need:
import numpy as np
from scipy import sparse
from scipy.sparse import linalg 
import matplotlib.pylab as plt
%matplotlib inline

A  truss element only has axial stiffness $\frac{AE}{L}$

The force-displacement relationship in its local reference system is:

$$
\begin{Bmatrix}f_1\\f_2\end{Bmatrix}=\frac{AE}{L}\begin{bmatrix}1&-1\\-1&1\end{bmatrix}\begin{Bmatrix}u_1\\u_2\end{Bmatrix}
$$

where $u_1$, $u_2$, $f_1$ and $f_2$ are all inline with the element axis.



<center><img src="https://drive.google.com/file/d/1Khxs9Fd7V23xrM6W0bbJrOT9X9FjID6P/view?usp=sharing" alt="files" style="width:500px"></center>

**_Figure 1: A truss element in its local reference system_**  
(see [13_truss_elements.ipynb](https://nbviewer.jupyter.org/github/jacojvr/Introductory-Finite-Elements/blob/a1431adaaaf5304a233e33e8967bde598790db9d/notebooks/13_truss_elements.ipynb) for the original)




## Constructing a case dictionary for a problem we want to solve


Consider we want to solve the displacements unknowns that mean the structure assembled using the two-elements below are in equilibrium...


<center><img src="https://nbviewer.jupyter.org/github/jacojvr/Introductory-Finite-Elements/blob/a1431adaaaf5304a233e33e8967bde598790db9d/notebooks/img/cercha2.png" alt="files" style="width:500px"></center>

**_Figure 2: Simple two truss structure _**  
(again see [13_truss_elements.ipynb](https://nbviewer.jupyter.org/github/jacojvr/Introductory-Finite-Elements/blob/a1431adaaaf5304a233e33e8967bde598790db9d/notebooks/13_truss_elements.ipynb) for the original)

...


There are different software packages or analytical approaches to solve this simple problem,  
depending on the FE package we use, there's a range of different input file formats we can choose from

### In this course (and notebooks / class examples):

* _We'll use a Python "dictionary of dictionaries"  to contain all of the relevant information in a structured format._

*  _To further have interchangablility with the CalculiX input deck format, we also won't allow 0 indexing of nodes or elements, so..._

### ...the lowest element or node number allowed is  1 , not  0  !!**

In [116]:
# Lets construct a case dictionary for the layout in Figure 2:
case = {
    # first define nodal coordinates { x,y } for each of the nodes
        'node': {1 : [ 0.,  0.], # node 1 : x= 0., y = 0.  ... node 0 in Figure 2, but zero indexing not allowed
                 2 : [ 3.,  0.], # node 2 : x= 3., y = 0.  ... node 1 in Figure 2
                 3 : [ 1.5,  1.5]}, # node 3 : x= 1.5, y = 1.5  ... node 2 in Figure 2
    # now define the elements (element sets defined by giving each a name) 
        'element': {'frame' : {'type': 't3d2', # the 2D Truss element consistent with CalculiX
                               1 : [ 1, 3 ], # elements and connectivity (e.g. element 1 connects node 1 and 3) 
                               2 : [ 2, 3 ]}},
    # define the FIXED displacement boundary conditions:
    # takes the form... (node, dof) : value
        'boundary' : {(1,1) : 0., # node 1, direction 1 (x) has a prescribed value of 0 (i.e. fixed in place)
                      (1,2) : 0.,
                      (2,1) : 0.,
                      (2,2) : 0.},
    # define materials used in the study, each material has a name and various properties associated with it
    # we'll use material keywords (i) elastic, (ii) plastic and (iii) density in this course...
    # this example material has only elastic behaviour
        'material' : {'example' : {'elastic' : [ 1.0, 0.0] }, # elastic properties [ Young's Modulus, Poisson's ratio]
                     },
    # define a section to model ... typically linking an element set with a material and defining the section thickness / area
        'section' : {0 : {'el_set' : 'frame', # element set
                          'material' : 'example', # uses a specific material defined
                          'area': 0.1}}, # has a specific thickness or area
    # define the step to solve...
    # this includes the time, loads and displacements active in a specific step
    # we could have multiple steps for e.g. a loading and unloading process if we include further steps...[2,3 etc.]
        'step' : {1 :  # time = [initial time step size, total step time, minimum step size, maximum step size]
                      {'time' : [1., 1., 1., 1.],
                       # define a concentrated / nodal load
                       # takes the form... (node, dof) : value
                       'cload' : {(3,1) : -1},   # node 3, direction 1 (x) has a concentrated load of -1 (at the end of the step time)
                       # ... will use this to further define PRESCRIBED displacement or DISTRIBUTED loads later
                       #
                       #
                       # also NOTE:
                       # We can also do prescribed displacements in the step definition later on...
                       # for this introduction and code to follow, we don't take prescribed displacements into account
                       
                      }}} 

In [117]:
# what are the coordinates of element 1?
el_nr = 1
#
# nodes associated with the element
el_nodes = case['element']['frame'][el_nr]
print(f'\nThe nodes associated with element {el_nr} is {el_nodes}')
#
# coordinates associated with those nodes
el_coords = np.array([case['node'][i] for i in el_nodes])
print('\nThe actual nodal coordinates are:')
print(el_coords)


The nodes associated with element 1 is [1, 3]

The actual nodal coordinates are:
[[0.  0. ]
 [1.5 1.5]]


## Now, some helper or utility functions:

## (UTILITY FUNCTION 1)
Here we define and test a function that takes a node and local degree of freedom tuple asd returns a global dof:

For our global system, note that Python uses 0 indexing, so:
 * Node 1, Direction 1 (x) will be global dof = 0
 * Node 1, Direction 2 (y) will be global dof = 1
 * There is no direction 3 (z) in 2D - so...
 * Node 2, Direction 1 (x) will be global dof = 3
 * ... and so on


In [118]:
# Here we define and test a function that takes a node and local degree of freedom tuple asnd returns a global dof:
#
# For our global system, note that Python uses 0 indexing, so:
# Node 1, Direction 1 will be global dof = 0
g_dof = lambda xd : 2*(xd[0]-1)+xd[1]-1

g_dof_print = lambda xd : print(f'\n**The global degree of freedom associated with node {xd[0]} in direction {xd[1]} is {g_dof(xd)}')

g_dof_print((1,1))
g_dof_print((1,2))
g_dof_print((3,1))
g_dof_print((15,2))


**The global degree of freedom associated with node 1 in direction 1 is 0

**The global degree of freedom associated with node 1 in direction 2 is 1

**The global degree of freedom associated with node 3 in direction 1 is 4

**The global degree of freedom associated with node 15 in direction 2 is 29




## (UTILITY FUNCTION 2)
Given an element definition, return all of the element global degrees of freedom

For an element with two nodes:
\[ 1 , 3 \]

and two local degree of freedom  
(x-direction) --> (1) and  
(y-direction) --> (2)  

The global degrees of freedom are mapped:  
(1,1) --> 0  
(1,2) --> 1  
(3,1) --> 4  
(3,2) --> 5


In [119]:
# Here we define and test a function that takes an element definition and
# returns a list of global degrees of freedom associated with the element

el_dof = lambda el_nodes : [ 2*(nd-1)+l_dof for nd in el_nodes for l_dof in [0,1]]

el_dof_print = lambda el_nodes : print(f'\n**The global degrees of freedom associated with element nodes\n\t{el_nodes}\nis \n\t{el_dof(el_nodes)}')

el_dof_print([1,3])
el_dof_print([2,5,9,12])


**The global degrees of freedom associated with element nodes
	[1, 3]
is 
	[0, 1, 4, 5]

**The global degrees of freedom associated with element nodes
	[2, 5, 9, 12]
is 
	[2, 3, 8, 9, 16, 17, 22, 23]



## NOW...

## Exporing / navigating the case dictionary...



In [120]:
# well construct the global stiffness over each section...

# in this case we only have section = 0
section_info = case['section'][0]
print('\nsection_info = \n',section_info)

# for an element stiffness matrix, we'll also need the section mataterial
section_material = case['material']
print('\nsection_material = \n',section_material)

# the material used in this section is:
material_info = section_material[section_info['material']]
print('\nmaterial_info = \n',material_info)


section_info = 
 {'el_set': 'frame', 'material': 'example', 'area': 0.1}

section_material = 
 {'example': {'elastic': [1.0, 0.0]}}

material_info = 
 {'elastic': [1.0, 0.0]}


## The element stiffness matrix

** _The definition of the truss element stiffness matrix as copied from [13_truss_elements.ipynb](https://nbviewer.jupyter.org/github/jacojvr/Introductory-Finite-Elements/blob/a1431adaaaf5304a233e33e8967bde598790db9d/notebooks/13_truss_elements.ipynb) is presented in this markdown cell...this notebook just replaces $\lambda$ in the original notebook with $Q$_.

To obtain the total stiffness from the structure it is now necessary to consider the stiffness contribution from all the elements in a common (**Global**) reference system.

Let:

* $U, F$ : Displacements degrees of freedom and forces in the global reference system.
* $u, f$ : Displacements degrees of freedom and forces in the local reference system.

and related by the rotational transformation matrix $Q$ like;

$$u=Q U.$$

Using the fact that the virtual energy of the forces upon imposition of virtual displacements is a reference-independent scalar quantity gives:

$$\delta U^TF=\delta u^Tf.$$

Using the first equation in the second yields;

$$\delta U^TF=\delta U^T Q^Tf$$

from which:

$$F=Q^Tf.$$

Now, conisdering the equilibrium relation for the element in the local system:

$$f=ku$$

where $k$ is the local stiffness matrix we can write:

$$\begin{array}{l}Q^Tf=Q^Tku\\Q^Tf=Q^TkQ U\\F=KU\end{array}$$

from which:

$$K=Q^TkQ$$

where $K$ is the stiffness matrix for the two-dimensional truss element in the global reference system. It must be observed that in the global reference system the element has two degrees of freedom per node.

In [121]:
# here implement the element stiffness function:

def el_stiffness(el_coords , section_info , material_info):
    
    # calculate element vector and length:
    vec = el_coords[1, :] - el_coords[0, :]
    length = np.linalg.norm(vec)
    #
    # Area contained in the section info
    A = section_info['area']
    # stiffness / Elastic modulus
    E = material_info['elastic'][0]
    #
    # elemet axial stiffness / in local reference frame:
    k = (A*E/length) * np.array([
        [1, -1],
        [-1, 1]])
    #
    # now transform that to cartesian coordinates:
    # first calculate the transformation matrix
    nx = vec[0]/length
    ny = vec[1]/length
    # 
    Q = np.array([
        [nx, ny , 0 , 0],
        [0,  0, nx , ny]])
    #
    #
    # now transform
    k_elem = np.dot(np.dot(Q.T, k), Q)
    return k_elem
    
    

In [122]:
# test it out with the element, section and material info we've already explored...

k_elem = el_stiffness(el_coords,section_info,material_info)
print(f'\nThe element stiffness matrix for element {el_nr} is:\n\n',k_elem)


The element stiffness matrix for element 1 is:

 [[ 0.02357023  0.02357023 -0.02357023 -0.02357023]
 [ 0.02357023  0.02357023 -0.02357023 -0.02357023]
 [-0.02357023 -0.02357023  0.02357023  0.02357023]
 [-0.02357023 -0.02357023  0.02357023  0.02357023]]




## The Global Stiffness Matrix

** _Juan Gomes has a great notebook to explain assembling a local element stiffness matrix into the global stiffness matrix [09_assembly.ipynb](https://nbviewer.jupyter.org/github/jacojvr/Introductory-Finite-Elements/blob/a1431adaaaf5304a233e33e8967bde598790db9d/notebooks/09_assembly.ipynb)._

In short...  
we need to map and add the elements of a local stiffness matrix $K_{element}^{(n,d)}$ to the global position $K_{global}^{(i,j)}$

The code below takes the 4x4 element stiffness matrix 

$$
\begin{bmatrix}1&2&3&4&\\5&6&7&8\\9&10&11&12\\13&14&15&16\end{bmatrix}
$$

... and maps them to the global system of equantions using the element global degrees of freedom \[ 0, 1, 4, 5\]  
the full system of equations has 6x degrees of freedom


In [123]:
# element level stiffness matrix
k_e = np.array([[1,2,3,4],
                [5,6,7,8],
                [9,10,11,12],
                [13,14,15,16]])
# in global degrees of freedom
pg = [0, 1, 4, 5] # can get this with our helper function "el_dof(el_nodes)" per element

# instead of a full / dense matrix representation we'll use a sparse matrix
# this means we don't use unnnecessary computer memory to store a whole bunch of zeros if we have a large system of equations
# it also speeds up calculations
#
# the values added to the sparse matrix are a flattened local stiffness matrix
values = k_e.flatten()
print('\n** adding the following local stiffness values :\n\t',values)
# in global row locations:
rows = [i for i in pg for _ in range(4)]
print('\n** to rows :\n\t',rows)
# in global columns locations:
cols = [i for _ in range(4) for i in pg]
print('\n** and columns :\n\t',cols)

# number of global degrees of freedom
n_dof = 6
#
# we'll use the csr_matrix function in scipy.sparse to assemble the global system of equations
k_global = sparse.csc_matrix((values,(rows,cols)),shape=(n_dof,n_dof));

print('\n** we get the sparse global matrix representation :\n',k_global)

print('\n** and the dense global matrix representation :\n',k_global.todense())


** adding the following local stiffness values :
	 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]

** to rows :
	 [0, 0, 0, 0, 1, 1, 1, 1, 4, 4, 4, 4, 5, 5, 5, 5]

** and columns :
	 [0, 1, 4, 5, 0, 1, 4, 5, 0, 1, 4, 5, 0, 1, 4, 5]

** we get the sparse global matrix representation :
   (0, 0)	1
  (1, 0)	5
  (4, 0)	9
  (5, 0)	13
  (0, 1)	2
  (1, 1)	6
  (4, 1)	10
  (5, 1)	14
  (0, 4)	3
  (1, 4)	7
  (4, 4)	11
  (5, 4)	15
  (0, 5)	4
  (1, 5)	8
  (4, 5)	12
  (5, 5)	16

** and the dense global matrix representation :
 [[ 1  2  0  0  3  4]
 [ 5  6  0  0  7  8]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 9 10  0  0 11 12]
 [13 14  0  0 15 16]]



## Assemble the global stiffness matrix (LHS) for the case dictionary

Impartant to note that we:
* loop through elements in all sections to assemble the system matrix
* get all the active system degrees of freedom
* remove prescribed degrees of freedom from the list of unknowns (i.e. don't solve for known degrees of freedom)

In [133]:
# initialise global sparse matrix lists
K_rows = []
K_cols = []
K_vals = []
#
# loop over sections defined in case
for s_name in case['section']:
    
    section_info = case['section'][s_name]
    elem_info = case['element'][section_info['el_set']]
    material_info = case['material'][section_info['material']]
    
    # get element type and numbers
    element_type = elem_info['type']
    element_nrs = [el_nr for el_nr in elem_info.keys() if type(el_nr) is int]
    
    # loop over elements:
    for el_nr in element_nrs:
        # element nodes
        el_nodes = elem_info[el_nr]
        # element coordinates:
        el_coords = np.array([case['node'][i] for i in el_nodes])
        # element stiffness:
        k_elem = el_stiffness(el_coords,section_info,material_info)
        # element global degrees of freedom
        pg = el_dof(el_nodes)
        # add to global stiffness matrix:
        K_rows += [i for i in pg for _ in range(4)]
        K_cols += [i for _ in range(4) for i in pg]
        K_vals += list(k_elem.flatten())

# what are the unique / active degrees of freedom?
all_dof = sorted(list(set(K_cols)))
# assemble the global stiffness matrix
k_global = sparse.csc_matrix((K_vals,(K_rows,K_cols)))

print('\n**The global stiffness matrix is:\n',np.round(k_global.todense(),4))


**The global stiffness matrix is:
 [[ 8.5355  3.5355 -3.5355 -3.5355 -5.      0.      0.      0.      0.
   0.    ]
 [ 3.5355  3.5355 -3.5355 -3.5355  0.      0.      0.      0.      0.
   0.    ]
 [-3.5355 -3.5355 12.0711  0.     -3.5355  3.5355 -5.      0.      0.
   0.    ]
 [-3.5355 -3.5355  0.      7.0711  3.5355 -3.5355  0.      0.      0.
   0.    ]
 [-5.      0.     -3.5355  3.5355 17.0711  0.     -3.5355 -3.5355 -5.
   0.    ]
 [ 0.      0.      3.5355 -3.5355  0.      7.0711 -3.5355 -3.5355  0.
   0.    ]
 [ 0.      0.     -5.      0.     -3.5355 -3.5355 12.0711  0.     -3.5355
   3.5355]
 [ 0.      0.      0.      0.     -3.5355 -3.5355  0.      7.0711  3.5355
  -3.5355]
 [ 0.      0.      0.      0.     -5.      0.     -3.5355  3.5355  8.5355
  -3.5355]
 [ 0.      0.      0.      0.      0.      0.      3.5355 -3.5355 -3.5355
   3.5355]]



## Assemble the global force vector (RHS)


In [125]:
case['step'][1]['cload']

{(3, 1): -1}

In [134]:
# get all the information for the solution step
step_info = case['step'][1]

# load information:
load_info = step_info['cload']

# initialise the global force vector (RHS)
F_rows = []
F_vals = []

# fill the correct load values into the index associated with the
# RHS global degrees of freedom
for dof,val in load_info.items():
    F_rows += [g_dof(dof)]
    F_vals += [val]

# global (sparse) force vector
f_global = sparse.csc_matrix((F_vals,(F_rows,[0]*F_rows.__len__())),(np.max(all_dof)+1,1))



## Solve the (unknown) displacements


In [135]:
# first, identify all of the degrees of freedom
# all_dof = already defined

# FIXED prescribed / known dof & Displacement (U):
dof_k = []
U_k = []
for dof,val in case['boundary'].items():
    dof_k += [g_dof(dof)]
    U_k += [val]
    
# STEP prescribed dof: 
step_info = case['step'][1]
if 'boundary' in step_info.keys():
    for dof,val in step_info['boundary'].items():
        dof_k += [g_dof(dof)]
        U_k += [val]
        
# UNKNOWN dof:
dof_u = [dof for dof in all_dof if dof not in dof_k]

print('system left with unknown dof :\n\t',dof_u)

system left with unknown dof :
	 [2, 3, 4, 5, 6, 7, 8]


In [137]:
# subset of equations to solve the unknown dof's:
K_u = k_global.T[dof_u].T[dof_u]
F_u = f_global[dof_u]

U_u = sparse.linalg.spsolve(K_u,F_u)


# Display solution

First - we need a helper function that converts global dof back into (node, direction)



## (UTILITY FUNCTION 3)
Here we define and test a function that takes a global degree of freedom and converts it into a tuple of (node, local dof):

For our global system, note that Python uses 0 indexing, so:
 * Node 1, Direction 1 (x) will be global dof = 0
 * Node 1, Direction 2 (y) will be global dof = 1
 * There is no direction 3 (z) in 2D - so...
 * Node 2, Direction 1 (x) will be global dof = 3
 * ... and so on


In [129]:
# Here we define and test a function that takes a global degree of freedom and returns a tuple of (node, local dof):
#
# For our global system, note that Python uses 0 indexing, so:
# Global dof = 0 will be Node 1, Direction 1
l_dof = lambda dof,dim=2 : (dof//dim+1, dof%dim+1)

l_dof_print = lambda dof,dim=2 : print(f'\n**The global degree of freedom {dof} in a {dim}-dimensional space is associated with node {dof//dim+1} in direction {dof%dim+1}')

l_dof_print(0)
l_dof_print(1)
l_dof_print(4)
l_dof_print(29)


**The global degree of freedom 0 in a 2-dimensional space is associated with node 1 in direction 1

**The global degree of freedom 1 in a 2-dimensional space is associated with node 1 in direction 2

**The global degree of freedom 4 in a 2-dimensional space is associated with node 3 in direction 1

**The global degree of freedom 29 in a 2-dimensional space is associated with node 15 in direction 2



## Print / Write out the solution:


In [138]:
print('\nSolution to the unknown truss structure displacements:')
for i in range(U_u.__len__()):
    n_d = l_dof(dof_u[i])
    val = U_u[i]
    xy = {1:'x',2:'y'}
    print(f'Node {n_d[0]} displaces {round(val,4)} in the {xy[n_d[1]]}-direction ')


Solution to the unknown truss structure displacements:
Node 2 displaces 0.2 in the x-direction 
Node 2 displaces -0.3414 in the y-direction 
Node 3 displaces 0.1 in the x-direction 
Node 3 displaces -0.5828 in the y-direction 
Node 4 displaces -0.0 in the x-direction 
Node 4 displaces -0.3414 in the y-direction 
Node 5 displaces 0.2 in the x-direction 


# Homework:

Report back in a short text document / pdf:


## Q1. Compare the solution you get in this notebook to that using Calculix:

1.1 Is there a difference in the solution?  
1.2 Looking at the "machine precision zero values" reported in Calculix (i.e. ~ 1e-16), why would Calculix not have exact zeros where we have it?  
(you may refer to the Calculix User Manual...how does Calculix "approximate" the T3D2 element?)

... here's the CCX input file:

```
**
**  MODEL A  SIMPLE TRUSS STRUCTURE
**
**
*NODE,NSET=Nall
**
1, 0.0, 0.0, 0.0
2, 3.0, 0.0, 0.0
3, 1.5, 1.5, 0.0
**
*ELEMENT,TYPE=T3D2,ELSET=EAll
1, 1, 3
2, 2, 3
**
*BOUNDARY
1,1,1
1,2,2
2,1,1
2,2,2
NALL,3,3
*MATERIAL,NAME=EXAMPLE
*ELASTIC
1.,0.
*SOLID SECTION,ELSET=EAll,MATERIAL=EXAMPLE
0.1
*STEP
*STATIC
*CLOAD
3, 1, -1.0
*Node PRINT,NSET=Nall
U,RF
*END STEP

```

## Q2. Solve the following truss structure problem :

** Please attach your CalculiX **input file** and Modified Notebook or **case dictionary** to your homework submission




In [132]:
# Lets construct a case dictionary for the layout:
case = {
    # first define nodal coordinates { x,y } for each of the nodes
        'node': {1:[0.0, 0.0, 0.0],
                 2:[1.0, 1.0, 0.0],
                 3:[2.0, 0.0, 0.0],
                 4:[3.0, 1.0, 0.0],
                 5:[4.0, 0.0, 0.0]},
    # now define the elements (element sets defined by giving each a name) 
        'element': {'frame' : {'type': 't3d2', # the 2D Truss element consistent with CalculiX
                               1 : [ 1, 2 ], 
                               2 : [ 1, 3 ], 
                               3 : [ 2, 3 ], 
                               4 : [ 2, 4 ], 
                               5 : [ 3, 4 ], 
                               6 : [ 3, 5 ], 
                               7 : [ 4, 5 ]}},
    # define the FIXED displacement boundary conditions:
    # takes the form... (node, dof) : value
        'boundary' : {(1,1) : 0.,
                      (1,2) : 0.,
                      (5,2) : 0.},
    # define materials used in the study, each material has a name and various properties associated with it
    # we'll use material keywords (i) elastic, (ii) plastic and (iii) density in this course...
    # this example material has only elastic behaviour
        'material' : {'example' : {'elastic' : [ 100.0, 0.0] }, # elastic properties [ Young's Modulus, Poisson's ratio]
                     },
    # define a section to model ... typically linking an element set with a material and defining the section thickness / area
        'section' : {0 : {'el_set' : 'frame', # element set
                          'material' : 'example', # uses a specific material defined
                          'area': 0.1}}, # has a specific thickness or area
    # define the step to solve...
    # this includes the time, loads and displacements active in a specific step
    # we could have multiple steps for e.g. a loading and unloading process if we include further steps...[2,3 etc.]
        'step' : {1 :  # time = [initial time step size, total step time, minimum step size, maximum step size]
                      {'time' : [1., 1., 1., 1.],
                       # define a concentrated / nodal load
                       # takes the form... (node, dof) : value
                       'cload' : {(3,2) : -1},   # node 3, direction 1 (x) has a concentrated load of -1 (at the end of the step time)
                       # ... will use this to further define PRESCRIBED displacement or DISTRIBUTED loads later
                       #
                       #
                       # also NOTE:
                       # We can also do prescribed displacements in the step definition later on...
                       # for this introduction and code to follow, we don't take prescribed displacements into account
                       
                      }}} 

```
**
**  MODEL A  SIMPLE TRUSS STRUCTURE
**
**
*NODE,NSET=Nall
**
1, 0.0, 0.0, 0.0
2, 1.0, 1.0, 0.0
3, 2.0, 0.0, 0.0
4, 3.0, 1.0, 0.0
5, 4.0, 0.0, 0.0
**
*ELEMENT,TYPE=T3D2,ELSET=EAll
1, 1, 2
2, 1, 3
3, 2, 3
4, 2, 4
5, 3, 4
6, 3, 5
7, 4, 5
**
*BOUNDARY
1,1,1
1,2,2
5,2,2
NALL,3,3
*MATERIAL,NAME=EXAMPLE
*ELASTIC
100.,0.
*SOLID SECTION,ELSET=EAll,MATERIAL=EXAMPLE
0.1
*STEP
*STATIC
*CLOAD
3, 2, -1.0
*Node PRINT,NSET=Nall
U,RF
*Node FILE,NSET=Nall
U
*El FILE,NSET=Nall
S
*END STEP
```